In [1]:
import sys
sys.path.append('../network')
import network
import pandas as pd
import copy


colors = {
        'c__Gammaproteobacteria': '#85D696', 
        'c__Alphaproteobacteria': '#D95F02', 
        'c__Clostridia': '#7EC9CE', 
        'c__Bacilli': '#FFC68A', 
        'c__Actinomycetia': '#CC99DB', 
        'c__Bacteroidia': '#999FDB', 
        'c__Cyanobacteriia': '#C0E481', 
        'other': '#AAAAAA'}

taxa_table = pd.read_csv('../data/taxonomy_df.tsv', sep='\t', header=0, index_col=None)


In [2]:
class_table = taxa_table[['g', 'c']]
class_table = class_table.drop_duplicates()
class_table.dropna(inplace=True, how='any')
# drop idx == g__
class_table = class_table[class_table['g'] != 'g__']
class_table = class_table.set_index('g')


In [4]:

colors_reverse = {}
colors_dict = {}
phy_dict = {}
for phy, color in colors.items():
    colors_dict[color] = 0
    colors_reverse[color] = phy
    phy_dict[phy] = []
total_num = class_table.shape[0]

for sp in class_table.index:
    phy = class_table.loc[sp, 'c']
    #print(phy, sp)
    # if sp duplicated in index
    if type(phy) == pd.core.series.Series or (phy not in colors.keys()):
        phy = 'other'
        class_table.loc[sp, 'c'] = 'other'
    colors_dict[colors[phy]] += 1
    phy_dict[phy].append(sp)

rnode = 5
width, origin, polar_df = network.assign_pos(total_num, rnode, margin=0)
class_table['g'] = class_table.index
class_table.drop_duplicates(inplace=True)
class_table[['g', 'c']].to_csv('class_genus.tsv', sep='\t', header=True, index=False)


In [5]:

def complete_df(layout, phy_dict, colors_reverse):
    col_order = ['id', 'theta', 'r', 'color', 'phy', 'bg_color']
    for idx in layout.index:
        color = layout.loc[idx, 'color'] 
        phy = colors_reverse[color]
        #print(color, phy)
        layout.loc[idx, 'id'] = phy_dict[phy].pop()
        layout.loc[idx, 'phy'] = phy
    layout['bg_color'] = layout['color']
    return layout[col_order]


In [6]:
# test ring_layout
#layout = network.ring_layout(colors_dict, polar_df)
#complete_df(layout, copy.deepcopy(phy_dict), colors_reverse).to_csv("ring_sp_layout.tsv", sep='\t', index=None)

# test sector_layout
layout = network.sector_layout(colors_dict, polar_df)
complete_df(layout, copy.deepcopy(phy_dict), colors_reverse).to_csv("sector_sp_layout.tsv", sep='\t', index=None)

# test scale_layout
#layout, seeds = network.scale_layout(colors_dict, polar_df, width)
#complete_df(layout, copy.deepcopy(phy_dict), colors_reverse).to_csv("scale_sp_layout.tsv", sep='\t', index=None)

with open('config.txt', 'w') as fp:
    fp.write("width = {}\norigin = {},{}\nr = {}".format(width, origin[0], origin[1], rnode))